In [ ]:
## 📊 Exploratory Data Analysis (EDA)

import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2
from tqdm import tqdm

# --- CONFIGURATION (from the original notebook) ---
TRAIN_ROOT = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/train_images"
MASK_ROOT = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/train_masks"

print("--- Starting Basic EDA ---")

# 1. Prepare Data List (same logic as in the notebook)
data_list = []
for root, _, files in os.walk(TRAIN_ROOT):
    for f in files:
        valid_extensions = ('.png', '.jpg', '.jpeg', '.tif', '.tiff', '.npy')
        if f.lower().endswith(valid_extensions):
            if 'forged' in root.lower():
                case_id = os.path.splitext(f)[0]
                img_path = os.path.join(root, f)
                mask_path = os.path.join(MASK_ROOT, f"{case_id}.npy")
                data_list.append({
                    'case_id': case_id,
                    'img_path': img_path,
                    'mask_path': mask_path
                })
full_df = pd.DataFrame(data_list)

# Filter for images with existing masks
if not full_df.empty:
    full_df['mask_exists'] = full_df['mask_path'].apply(os.path.exists)
    eda_df = full_df[full_df['mask_exists']].drop(columns=['mask_exists']).reset_index(drop=True)
else:
    eda_df = pd.DataFrame(columns=['case_id', 'img_path', 'mask_path'])


# 2. File and Case Counts
print(f"\nTotal potential images found in TRAIN_ROOT: {len(data_list)}")
print(f"Total valid image/mask pairs for training (Forged Cases): {len(eda_df)}")

# 3. Image and Mask Size Distribution
if not eda_df.empty:
    img_heights, img_widths = [], []
    mask_pixels = [] # Count of non-zero pixels in the mask
    
    print("\nAnalyzing image and mask dimensions/forgery area...")
    for index, row in tqdm(eda_df.iterrows(), total=len(eda_df)):
        try:
            # Read Image
            img = cv2.imread(row['img_path'])
            if img is not None and img.size > 0:
                h, w = img.shape[:2]
                img_heights.append(h)
                img_widths.append(w)

            # Read Mask (npy file)
            mask = np.load(row['mask_path'])
            # Assuming the forgery area is represented by non-zero pixels
            mask_pixels.append(np.sum(mask > 0)) 

        except Exception as e:
            # Handle files that can't be read (e.g., non-image .npy files not handled by cv2.imread)
            pass

    print("\n--- Image Dimension Stats ---")
    print(f"Unique Image Widths: {sorted(list(set(img_widths)))[:5]}{'...' if len(set(img_widths)) > 5 else ''}")
    print(f"Unique Image Heights: {sorted(list(set(img_heights)))[:5]}{'...' if len(set(img_heights)) > 5 else ''}")
    print(f"Mean Image Dimensions (H x W): {np.mean(img_heights):.0f} x {np.mean(img_widths):.0f}")

    # 4. Forgery Area Analysis
    mask_pixels = np.array(mask_pixels)
    forged_cases_with_area = np.sum(mask_pixels > 0)
    total_forgery_area = np.sum(mask_pixels)
    
    print("\n--- Forgery Area Stats ---")
    print(f"Total cases with non-zero forgery area: {forged_cases_with_area} / {len(eda_df)}")
    print(f"Mean Forgery Pixel Count per forged image: {np.mean(mask_pixels[mask_pixels > 0]):.0f} (pixels)")
    print(f"Maximum Forgery Pixel Count: {np.max(mask_pixels)} (pixels)")

    # 5. Visualization: Forgery Area Distribution (First 100 cases for quick view)
    plt.figure(figsize=(12, 5))
    plt.bar(range(min(100, len(mask_pixels))), mask_pixels[:100])
    plt.title('Forgery Pixel Count (First 100 Forged Samples)')
    plt.xlabel('Sample Index')
    plt.ylabel('Forged Pixel Count (Area)')
    plt.show()

else:
    print("🛑 EDA Skipped: The dataframe is empty. Check TRAIN_ROOT and MASK_ROOT paths.")

print("\n--- EDA Complete ---")

In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
from warnings import filterwarnings

filterwarnings('ignore') # Suppress warnings

# --- CONFIGURATION (from the original notebook) ---
TARGET_SIZE = 256
TRAIN_ROOT = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/train_images"
MASK_ROOT = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/train_masks"

# Replicate compute_ela for feature analysis
def compute_ela(img_path, quality=95, scale=10):
    # ... (omitted for brevity, assume the original function is available)
    # The original notebook's ELA function is used here.
    img = cv2.imread(img_path)
    if img is None or img.size == 0:
        try:
            img_data = np.load(img_path)
            if img_data.ndim == 3: img = cv2.cvtColor(img_data, cv2.COLOR_RGB2BGR)
            elif img_data.ndim == 2: img = cv2.cvtColor(img_data, cv2.COLOR_GRAY2BGR)
        except Exception: return np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)

    if img is None or img.size == 0:
        return np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)

    img_resized = cv2.resize(img, (TARGET_SIZE, TARGET_SIZE))
    temp_path = f"/tmp/temp_ela_{os.path.basename(img_path)}.jpg" # Simplified temp_path
    try:
        # Use a consistent quality setting (95)
        cv2.imwrite(temp_path, img_resized, [cv2.IMWRITE_JPEG_QUALITY, quality]) 
        compressed_img = cv2.imread(temp_path)
        if compressed_img is None: return np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)
        error = np.abs(img_resized.astype(np.float32) - compressed_img.astype(np.float32))
        ela_feature_2d = np.mean(error, axis=2) * scale # Scale by 10 as in the notebook
    finally:
        if os.path.exists(temp_path): os.remove(temp_path)
    return cv2.resize(ela_feature_2d, (TARGET_SIZE, TARGET_SIZE), interpolation=cv2.INTER_LINEAR).astype(np.float32)

# Load the filtered DataFrame (assuming the prior EDA cell's 'eda_df' is available or recreate it)
data_list = []
for root, _, files in os.walk(TRAIN_ROOT):
    for f in files:
        valid_extensions = ('.png', '.jpg', '.jpeg', '.tif', '.tiff', '.npy')
        if f.lower().endswith(valid_extensions) and 'forged' in root.lower():
            case_id = os.path.splitext(f)[0]
            mask_path = os.path.join(MASK_ROOT, f"{case_id}.npy")
            if os.path.exists(mask_path):
                data_list.append({'img_path': os.path.join(root, f), 'mask_path': mask_path})
eda_df = pd.DataFrame(data_list)

print("--- Starting Advanced EDA (Imbalance & Feature Check) ---")

if eda_df.empty:
    print("🛑 EDA Skipped: Data frame is empty.")
else:
    total_pixels = 0
    forgery_pixels = 0
    ela_values, rgb_means = [], []

    # Process only the first 50 images to speed up ELA computation for EDA
    for index, row in tqdm(eda_df.head(50).iterrows(), total=len(eda_df.head(50)), desc="Processing samples"):
        try:
            # 1. Image and Mask Load
            rgb_image = cv2.cvtColor(cv2.imread(row['img_path']), cv2.COLOR_BGR2RGB)
            if rgb_image is None or rgb_image.size == 0: continue
                
            mask = np.load(row['mask_path'])
            if mask.ndim > 2: mask = mask[:, :, 0]
            
            # 2. Imbalance Check (Use original sizes for best estimate)
            h, w = rgb_image.shape[:2]
            total_pixels += h * w
            forgery_pixels += np.sum(mask > 0)
            
            # 3. ELA Feature Check (Use 256x256 resized data)
            ela_feature = compute_ela(row['img_path'])
            ela_values.extend(ela_feature.flatten())
            
            # RGB feature check (resize/normalize similar to training)
            rgb_resized = cv2.resize(rgb_image, (TARGET_SIZE, TARGET_SIZE)) / 255.0
            rgb_means.extend(rgb_resized.mean(axis=2).flatten())

        except Exception as e:
            # print(f"Warning: Could not process {row['img_path']}: {e}")
            continue

    # --- Analysis 1: Imbalance Ratio ---
    if total_pixels > 0:
        imbalance_ratio = (forgery_pixels / total_pixels) * 100
        print(f"\n--- Imbalance Ratio (Forged Pixels) ---")
        print(f"Total Pixels Sampled: {total_pixels:,}")
        print(f"Forged Pixels Sampled: {forgery_pixels:,}")
        print(f"Forgery Imbalance Ratio: **{imbalance_ratio:.2f}%** (Positive Class)")
    
    # --- Analysis 2: ELA Feature Distribution vs. RGB ---
    if ela_values:
        ela_values = np.array(ela_values)
        rgb_means = np.array(rgb_means)

        print(f"\n--- ELA Feature Distribution (Scaled by 10) ---")
        print(f"ELA Feature Mean: {np.mean(ela_values):.4f}")
        print(f"ELA Feature Std Dev: {np.std(ela_values):.4f}")
        print(f"RGB Mean (Normalized): {np.mean(rgb_means):.4f}")

        plt.figure(figsize=(12, 5))
        plt.hist(ela_values, bins=50, alpha=0.6, label='ELA Feature (Scaled)', color='red')
        plt.title('Distribution of ELA Feature Values')
        plt.xlabel('ELA Value (0 to ~2550)')
        plt.ylabel('Frequency')
        plt.legend()
        plt.show()
        
        # This histogram helps visualize if ELA is predominantly zero or clustered.

print("\n--- Advanced EDA Complete ---")

In [ ]:
import matplotlib.pyplot as plt
import cv2
import os

# Define the file path
TEST_IMAGE_PATH = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/test_images/45.png"

print(f"Attempting to load image: {TEST_IMAGE_PATH}")

if not os.path.exists(TEST_IMAGE_PATH):
    print("🛑 ERROR: The file path was not found. Please ensure the Kaggle competition data is mounted correctly.")
else:
    # Load the image using OpenCV (loads as BGR)
    img = cv2.imread(TEST_IMAGE_PATH)
    
    if img is None:
        print("🛑 ERROR: Could not read the image file.")
    else:
        # Convert the image from BGR (OpenCV default) to RGB (Matplotlib default)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Plot the image
        plt.figure(figsize=(10, 8))
        plt.imshow(img_rgb)
        plt.title(f"Test Image 45 (Dimensions: {img.shape[0]}x{img.shape[1]})")
        plt.axis('off') # Hide axes for a cleaner image view
        plt.show()

In [ ]:
import numpy as np
import cv2
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
import time
from torch.optim.lr_scheduler import ReduceLROnPlateau
import warnings
from warnings import filterwarnings

# Suppress the specific UserWarning from the LR scheduler
warnings.filterwarnings("ignore", category=UserWarning, module="torch.optim.lr_scheduler")
filterwarnings('ignore')

# --- CONFIGURATION --
TARGET_SIZE = 256
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 8
# REDUCED EPOCHS to 3 (was 10)
EPOCHS = 2
LEARNING_RATE = 1e-4

# --- PATHS (CORRECTED FOR KAGGLEHUB CACHE) ---
TRAIN_ROOT = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/train_images"
MASK_ROOT = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/train_masks"
MODEL_SAVE_PATH = "/tmp/model_new_scratch.pth"

# --- UTILITY FUNCTIONS ---
def compute_ela(img_path, quality=95, scale=10):
    img = cv2.imread(img_path)
    if img is None or img.size == 0:
        try:
            img_data = np.load(img_path)
            if img_data.ndim == 3:
                img = cv2.cvtColor(img_data, cv2.COLOR_RGB2BGR)
            elif img_data.ndim == 2:
                img = cv2.cvtColor(img_data, cv2.COLOR_GRAY2BGR)
        except Exception:
            return np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)

    if img is None or img.size == 0:
        return np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)

    img_resized = cv2.resize(img, (TARGET_SIZE, TARGET_SIZE))
    temp_path = f"/tmp/temp_ela_{os.path.basename(img_path)}_{time.time()}.jpg"
    try:
        cv2.imwrite(temp_path, img_resized, [cv2.IMWRITE_JPEG_QUALITY, quality])
        compressed_img = cv2.imread(temp_path)
        if compressed_img is None: return np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)
        error = np.abs(img_resized.astype(np.float32) - compressed_img.astype(np.float32))
        ela_feature_2d = np.mean(error, axis=2) * scale
    finally:
        if os.path.exists(temp_path): os.remove(temp_path)
    return cv2.resize(ela_feature_2d, (TARGET_SIZE, TARGET_SIZE),
                      interpolation=cv2.INTER_LINEAR).astype(np.float32)

class DiceLoss(nn.Module):
    def __init__(self, smooth=1.0):
        super(DiceLoss, self).__init__()
        self.smooth = smooth
    def forward(self, pred, target):
        pred = pred.contiguous().view(-1)
        target = target.contiguous().view(-1)
        intersection = (pred * target).sum()
        dice = (2. * intersection + self.smooth) / (pred.sum() + target.sum() + self.smooth)
        return 1 - dice

# Hybrid Loss combining Dice and BCE for stable training
class HybridLoss(nn.Module):
    def __init__(self, dice_weight=0.5):
        super(HybridLoss, self).__init__()
        self.dice_loss = DiceLoss()
        self.bce_loss = nn.BCELoss()
        self.dice_weight = dice_weight

    def forward(self, pred, target):
        dice = self.dice_loss(pred, target)
        bce = self.bce_loss(pred, target)
        return self.dice_weight * dice + (1 - self.dice_weight) * bce

# U-Net architecture
class UNet(nn.Module):
    def __init__(self, in_channels=4, num_classes=1):
        super().__init__()
        def block(in_c, out_c):
            return nn.Sequential(
                nn.Conv2d(in_c, out_c, 3, 1, 1), nn.ReLU(),
                nn.Dropout(p=0.2),
                nn.Conv2d(out_c, out_c, 3, 1, 1), nn.ReLU()
            )

        self.enc1 = block(in_channels, 64)
        self.enc2 = block(64, 128)
        self.bottleneck = block(128, 256)
        self.upconv2 = nn.ConvTranspose2d(256, 128, 2, 2)
        self.dec2 = block(128 + 128, 128)
        self.upconv1 = nn.ConvTranspose2d(128, 64, 2, 2)
        self.dec1 = block(64 + 64, 64)
        self.final_conv = nn.Conv2d(64, num_classes, 1)

    def forward(self, x):
        e1 = self.enc1(x)
        p1 = F.max_pool2d(e1, 2)
        e2 = self.enc2(p1)
        p2 = F.max_pool2d(e2, 2)
        b = self.bottleneck(p2)
        d2 = self.upconv2(b)
        d2 = torch.cat((d2, e2), dim=1)
        d2 = self.dec2(d2)
        d1 = self.upconv1(d2)
        d1 = torch.cat((d1, e1), dim=1)
        d1 = self.dec1(d1)
        return torch.sigmoid(self.final_conv(d1))

class ForgeryDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = row['img_path']
        mask_path = row['mask_path']

        # --- Load Image ---
        rgb_image = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)

        if rgb_image is None or rgb_image.size == 0:
            try:
                img_data = np.load(img_path)
                if img_data.ndim == 3:
                    rgb_image = img_data
                elif img_data.ndim == 2:
                    rgb_image = cv2.cvtColor(img_data, cv2.COLOR_GRAY2RGB)
            except Exception as e:
                raise RuntimeError(f"Failed to load image from {img_path}: {e}")

        # --- Load Mask ---
        try:
            mask = np.load(mask_path)
            if mask.ndim > 2:
                mask = mask[:, :, 0]
        except Exception as e:
            raise RuntimeError(f"Failed to load mask from {mask_path}: {e}")

        ela_feature_2d = compute_ela(img_path)

        # Resize all features
        rgb_image_resized = cv2.resize(rgb_image, (TARGET_SIZE, TARGET_SIZE))
        ela_feature_resized = cv2.resize(ela_feature_2d, (TARGET_SIZE, TARGET_SIZE))

        # Use INTER_NEAREST for binary mask resizing
        mask_resized = cv2.resize(mask.astype(np.uint8), (TARGET_SIZE, TARGET_SIZE), interpolation=cv2.INTER_NEAREST)

        # Stack RGB (3) and ELA (1) for a 4-channel input
        ela_feature_3d = np.expand_dims(ela_feature_resized, axis=-1)
        stacked_input = np.concatenate([rgb_image_resized, ela_feature_3d], axis=-1)

        # Convert to PyTorch tensors and normalize
        image = torch.tensor(stacked_input.transpose(2, 0, 1) / 255.0, dtype=torch.float32)
        mask = torch.tensor(mask_resized / 255.0, dtype=torch.float32).unsqueeze(0)

        return image, mask

def train_model(model, train_loader, val_loader, epochs=EPOCHS, save_path=MODEL_SAVE_PATH):
    # Use HybridLoss
    criterion = HybridLoss(dice_weight=0.5)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    # Scheduler with patience=2 (unchanged)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

    best_val_loss = float('inf')

    model.to(DEVICE)
    print(f"Starting training on {DEVICE} for {epochs} epochs...")

    for epoch in range(epochs):
        model.train()
        train_loss_sum = 0

        for inputs, targets in tqdm(train_loader, desc=f"Epoch {epoch+1} Training"):
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss_sum += loss.item()

        avg_train_loss = train_loss_sum / len(train_loader)

        # Validation Phase
        model.eval()
        val_loss_sum = 0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss_sum += loss.item()

        avg_val_loss = val_loss_sum / len(val_loader)

        # Scheduler Step
        scheduler.step(avg_val_loss)

        print(f"Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), save_path)
            print(f"Model saved successfully to {save_path}. (New best Val Loss: {best_val_loss:.4f})\n")

        current_lr = optimizer.param_groups[0]['lr']
        print(f"Current Learning Rate: {current_lr:.6f}")


# --- MAIN EXECUTION BLOCK ---
if __name__ == '__main__':

    print("Preparing training data paths...\n")

    data_list = []

    # Recursively walk through the TRAIN_ROOT to find all image files
    for root, _, files in os.walk(TRAIN_ROOT):
        for f in files:
            valid_extensions = ('.png', '.jpg', '.jpeg', '.tif', '.tiff', '.npy')

            if f.lower().endswith(valid_extensions):
                # Only process files in the 'forged' subdirectory, as only they have masks
                if 'forged' in root.lower():
                    case_id = os.path.splitext(f)[0]
                    img_path = os.path.join(root, f)

                    # Use .npy for the mask extension
                    mask_path = os.path.join(MASK_ROOT, f"{case_id}.npy")

                    data_list.append({
                        'case_id': case_id,
                        'img_path': img_path,
                        'mask_path': mask_path
                    })

    if not data_list:
        full_df = pd.DataFrame(columns=['case_id', 'img_path', 'mask_path'])
    else:
        full_df = pd.DataFrame(data_list)

    if not full_df.empty:
        # Final check: Keep only images that have a corresponding mask file
        full_df['mask_exists'] = full_df['mask_path'].apply(os.path.exists)
        full_df = full_df[full_df['mask_exists']].drop(columns=['mask_exists']).reset_index(drop=True)

    if full_df.empty:
        print("🛑 FATAL ERROR: No valid image/mask pairs found in the input paths. Cannot train. (Check file extensions/paths again)")
    else:
        print(f"✅ Found {len(full_df)} valid forged samples for training.")

        # Split data
        train_df, val_df = train_test_split(full_df, test_size=0.1, random_state=42)

        # Create DataLoaders
        train_dataset = ForgeryDataset(train_df)
        val_dataset = ForgeryDataset(val_df)
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

        # Instantiate model (4 input channels: 3 RGB + 1 ELA)
        model = UNet(in_channels=4)

        # START TRAINING
        train_model(model, train_loader, val_loader)

        print("\n✅ TRAINING COMPLETE. The trained model is saved and ready for inference.")

In [ ]:
# ============================================================
# efficientnet_ela_inference_offline.py (Kaggle-friendly)
# ============================================================

import os
import time
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import warnings
warnings.filterwarnings('ignore')

# ---------------- CONFIG ----------------
TARGET_SIZE = 256
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 8
FIXED_THRESHOLD = 0.45
MIN_FORGERY_AREA = 32

TEST_IMAGE_ROOT = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/test_images"
SAMPLE_SUBMISSION_FILE = "/kaggle/input/recodai-luc-scientific-image-forgery-detection/sample_submission.csv"
MODEL_PATH = "/tmp/model_new_scratch.pth"   # change if you have a checkpoint
OUTPUT_FILENAME = "submission.csv"

# ============================================================
# ---------------- UTIL: ELA ----------------
# ============================================================
def compute_ela(img_path, quality=95, scale=10):
    """
    Returns a TARGET_SIZE x TARGET_SIZE single-channel ELA image (float32).
    Robust to jpg/png/npy; returns zeros if unreadable.
    """
    img = None
    try:
        img = cv2.imread(img_path)
    except Exception:
        img = None

    if img is None or img.size == 0:
        try:
            arr = np.load(img_path)
            if arr.ndim == 3 and arr.shape[2] == 3:
                img = cv2.cvtColor(arr.astype(np.uint8), cv2.COLOR_RGB2BGR)
            elif arr.ndim == 2:
                img = cv2.cvtColor(arr.astype(np.uint8), cv2.COLOR_GRAY2BGR)
        except Exception:
            img = None

    if img is None or img.size == 0:
        return np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)

    img_resized = cv2.resize(img, (TARGET_SIZE, TARGET_SIZE))
    tmp_jpg = f"/tmp/ela_tmp_{os.getpid()}_{time.time()}.jpg"
    try:
        cv2.imwrite(tmp_jpg, img_resized, [int(cv2.IMWRITE_JPEG_QUALITY), quality])
        comp = cv2.imread(tmp_jpg)
        if comp is None:
            return np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)
        error = np.abs(img_resized.astype(np.float32) - comp.astype(np.float32))
        ela = np.mean(error, axis=2) * scale
    finally:
        if os.path.exists(tmp_jpg):
            try:
                os.remove(tmp_jpg)
            except Exception:
                pass

    ela_resized = cv2.resize(ela, (TARGET_SIZE, TARGET_SIZE), interpolation=cv2.INTER_LINEAR)
    return ela_resized.astype(np.float32)

# ============================================================
# ---------------- DATAFRAME CREATION ----------------
# ============================================================
def create_test_df_robust(test_image_root, sample_submission_path):
    df = pd.read_csv(sample_submission_path)
    df['case_id'] = df['case_id'].astype(str)
    present = {}
    if os.path.exists(test_image_root):
        for root, _, files in os.walk(test_image_root):
            for f in files:
                if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff', '.npy')):
                    present[os.path.splitext(f)[0]] = os.path.join(root, f)
    df['img_path'] = df['case_id'].map(present)
    df['img_path'] = df['img_path'].fillna('MISSING_FILE')
    return df[['case_id', 'img_path']]

# ============================================================
# ---------------- RLE ENCODING ----------------
# ============================================================
def rle_encode(mask):
    """Run-length encoding using column-major (Fortran) order — Kaggle standard."""
    if mask.sum() == 0:
        return "authentic"
    pixels = mask.astype(np.uint8).flatten(order='F')
    padded = np.pad(pixels, (1,1), mode='constant', constant_values=0)
    changes = np.where(padded[1:] != padded[:-1])[0]
    runs = changes.reshape(-1,2)
    starts = runs[:,0] + 1
    lengths = runs[:,1] - runs[:,0]
    return ' '.join(str(s) + ' ' + str(l) for s, l in zip(starts, lengths))

# ============================================================
# ---------------- MODEL DEFINITION ----------------
# ============================================================
class EffNetB0_ELA_Segmenter(nn.Module):
    def __init__(self, pretrained=False):  # <-- pretrained=False (offline safe)
        super().__init__()
        # Load EfficientNet without internet download
        self.backbone = models.efficientnet_b0(weights=None)

        # Modify first conv to accept 4 channels (RGB + ELA)
        first_conv = self.backbone.features[0][0]
        out_ch = first_conv.out_channels
        k = first_conv.kernel_size
        new_conv = nn.Conv2d(4, out_ch, kernel_size=k, stride=first_conv.stride,
                             padding=first_conv.padding, bias=False)

        with torch.no_grad():
            old_w = first_conv.weight.data.clone()
            new_w = torch.zeros((out_ch, 4, k[0], k[1]))
            new_w[:, :3, :, :] = old_w
            new_w[:, 3:4, :, :] = old_w.mean(dim=1, keepdim=True)
            new_conv.weight.data.copy_(new_w)
        self.backbone.features[0][0] = new_conv

        encoder_out_channels = 1280
        self.decoder = nn.Sequential(
            nn.Conv2d(encoder_out_channels, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
            nn.Conv2d(256, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 1, kernel_size=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        feats = self.backbone.features(x)
        mask = self.decoder(feats)
        mask_up = F.interpolate(mask, size=(TARGET_SIZE, TARGET_SIZE),
                                mode='bilinear', align_corners=False)
        return mask_up

# ============================================================
# ---------------- IMAGE TO TENSOR ----------------
# ============================================================
def image_to_tensor_rgb_ela(img_path):
    try:
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    except Exception:
        img = None
    if img is None or img.size == 0:
        try:
            arr = np.load(img_path)
            if arr.ndim == 2:
                arr = cv2.cvtColor(arr.astype(np.uint8), cv2.COLOR_GRAY2BGR)
            elif arr.ndim == 3 and arr.shape[2] == 3:
                arr = arr.astype(np.uint8)
            img = arr[:, :, ::-1]
        except Exception:
            rgb = np.zeros((TARGET_SIZE, TARGET_SIZE, 3), dtype=np.uint8)
            ela = np.zeros((TARGET_SIZE, TARGET_SIZE), dtype=np.float32)
            tensor = np.concatenate([rgb.transpose(2,0,1).astype(np.float32)/255.0,
                                     ela[np.newaxis, :]], axis=0)
            return torch.from_numpy(tensor).float()

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_rgb = cv2.resize(img_rgb, (TARGET_SIZE, TARGET_SIZE))
    ela = compute_ela(img_path)

    rgb_norm = img_rgb.astype(np.float32) / 255.0
    ela_norm = ela.astype(np.float32)
    if ela_norm.max() > 0:
        ela_norm = (ela_norm - ela_norm.min()) / (ela_norm.max() - ela_norm.min() + 1e-8)
    stacked = np.concatenate([rgb_norm.transpose(2,0,1), ela_norm[np.newaxis, ...]], axis=0)
    return torch.from_numpy(stacked).float()

# ============================================================
# ---------------- INFERENCE ----------------
# ============================================================
def run_inference(model_path=None, tta=True):
    device = DEVICE
    model = EffNetB0_ELA_Segmenter(pretrained=False).to(device)

    # Try loading weights if available
    if model_path and os.path.exists(model_path):
        state = torch.load(model_path, map_location=device)
        model_state = model.state_dict()
        compatible_state = {k: v for k, v in state.items()
                            if k in model_state and v.shape == model_state[k].shape}
        model_state.update(compatible_state)
        model.load_state_dict(model_state, strict=False)
        print(f"✅ Loaded compatible weights ({len(compatible_state)} matched layers).")
    else:
        print("⚠️ No checkpoint found or incompatible — using random weights.")

    model.eval()
    test_df = create_test_df_robust(TEST_IMAGE_ROOT, SAMPLE_SUBMISSION_FILE)
    results = []

    batch_imgs = []
    case_ids = []

    def flush_batch(batch_imgs, case_ids):
        if len(batch_imgs) == 0:
            return []
        x = torch.stack(batch_imgs, dim=0).to(device)
        with torch.no_grad():
            preds = model(x)
            if tta:
                x_flip = torch.flip(x, dims=[3])
                preds_flip = model(x_flip)
                preds_flip = torch.flip(preds_flip, dims=[3])
                preds = (preds + preds_flip) / 2.0
            preds_np = preds.squeeze(1).cpu().numpy()
        encs = []
        for p in preds_np:
            mask = (p > FIXED_THRESHOLD).astype(np.uint8)
            if mask.sum() < MIN_FORGERY_AREA:
                mask[:] = 0
            encs.append(rle_encode(mask))
        return encs

    for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Inference"):
        cid = row['case_id']
        path = row['img_path']
        case_ids.append(cid)
        if path == 'MISSING_FILE' or not os.path.exists(path):
            results.append("authentic")
            continue
        t = image_to_tensor_rgb_ela(path)
        batch_imgs.append(t)
        if len(batch_imgs) >= BATCH_SIZE:
            results.extend(flush_batch(batch_imgs, case_ids))
            batch_imgs, case_ids = [], []

    if len(batch_imgs) > 0:
        results.extend(flush_batch(batch_imgs, case_ids))

    submission_df = pd.DataFrame({
        'case_id': test_df['case_id'],
        'encoded_pixels': results
    })
    submission_df.to_csv(OUTPUT_FILENAME, index=False)
    print(f"\n✅ Created {OUTPUT_FILENAME} with {len(submission_df)} rows at {pd.Timestamp.now()}")
    return submission_df

# ============================================================
# ---------------- RUN ----------------
# ============================================================
if __name__ == "__main__":
    submission = run_inference(model_path=MODEL_PATH, tta=True)


In [ ]:
!cat submission.csv

In [ ]:
def validate_and_print_rle(submission_df):
    # Validate the submission format
    print("Submission columns:", submission_df.columns.tolist())
    
    authentic_count = submission_df['encoded_pixels'].apply(lambda x: x == 'authentic').sum()
    rle_rows = submission_df[submission_df['encoded_pixels'] != 'authentic']
    
    print(f"Total rows: {len(submission_df)}")
    print(f"Authentic count: {authentic_count}")
    print(f"Forgery (RLE) count: {len(rle_rows)}")
    
    # Show first few RLE entries
    print("\nSample RLE rows:")
    print(rle_rows.head())

# Use it
submission_df = pd.read_csv("submission.csv")
validate_and_print_rle(submission_df)


In [ ]:
submission_df = pd.read_csv("submission.csv")
validate_and_print_rle(submission_df)